In [1]:
#test the reshaping behavior of the Cholesky vectors
import numpy as np
import scipy
from pyscf import gto, ao2mo, lo, tools

def read_fcidump(fname, norb):
    """

    :param fname: electron integrals dumped by pyscf
    :param norb: number of orbitals
    :return: electron integrals for 2nd quantization with chemist's notation
    """
    v2e = np.zeros((norb, norb, norb, norb))
    h1e = np.zeros((norb, norb))

    with open(fname, "r") as f:
        lines = f.readlines()
        for line, info in enumerate(lines):
            if line < 4:
                continue
            line_content = info.split()
            integral = float(line_content[0])
            p, q, r, s = [int(i_index) for i_index in line_content[1:5]]
            if r != 0:
                # v2e[p,q,r,s] is with chemist notation (pq|rs)=(qp|rs)=(pq|sr)=(qp|sr)
                v2e[p-1, q-1, r-1, s-1] = integral
                v2e[q-1, p-1, r-1, s-1] = integral
                v2e[p-1, q-1, s-1, r-1] = integral
                v2e[q-1, p-1, s-1, r-1] = integral
            elif p != 0:
                h1e[p-1, q-1] = integral
                h1e[q-1, p-1] = integral
            else:
                nuc = integral
    return h1e, v2e, nuc

def hamiltonian_integral(mol):
    # 1e & 2e integrals
    s_mat = mol.intor('int1e_ovlp')
    ao_coeff = lo.orth.lowdin(s_mat)
    norb = ao_coeff.shape[0]
    import tempfile
    ftmp = tempfile.NamedTemporaryFile()
    tools.fcidump.from_mo(mol, ftmp.name, ao_coeff)
    h1e, eri, nuc = read_fcidump(ftmp.name, norb)
    # Cholesky decomposition
    v2e = eri.reshape((norb**2, -1))
    u, s, v = scipy.linalg.svd(v2e)
    l_tensor = u * np.sqrt(s)
    l_tensor = l_tensor.T
    l_tensor = l_tensor.reshape(l_tensor.shape[0], norb, norb)
    return h1e, eri, nuc, l_tensor

def cholesky(eri, nao):
    '''eri: 4d array, (nao, nao, nao, nao), chemist notation, i.e. (ij|kl) = <ik|jl>'''
    Vij = eri.reshape((nao**2, -1))
    print("Vij = ", Vij)
    try:
        L = scipy.linalg.cholesky(Vij, lower = True)
        ltensor = L.reshape((-1, nao, nao))
    except scipy.linalg.LinAlgError:
        # it happens, since the eri is not numerically positive definite
        e, v = scipy.linalg.eigh(Vij)
        print('e = ', e)
        print('v = ', v)
        # without selection of eigenvalues
        #L = v @ np.diag(np.sqrt(np.abs(e)))
        # with selection of eigenvalues
        idx = e > 1e-10
        L = (v[:,idx] * np.sqrt(e[idx]))
        L = np.flip(L, 1)
        ltensor = L.T.reshape((-1, nao, nao))
    return ltensor

mol = gto.Mole()
mol.build(
    atom = 'H 0 0 0; F 0 0 1.1',  # in Angstrom
    basis = 'ccpvdz',
    symmetry = True,
)

ovlp_mat = mol.intor('int1e_ovlp')
X = lo.orth.lowdin(ovlp_mat)
myeri = mol.ao2mo(X)
myeri = ao2mo.restore(1, np.asarray(myeri), mol.nao)

#print the results
h1e, eri, nuc, l_tensor = hamiltonian_integral(mol)
print(eri.shape)
v2e = eri.reshape((mol.nao**2, -1))
print(myeri.shape)
Vij = myeri.reshape((mol.nao**2, -1))
print(np.linalg.norm(Vij.T - Vij))
myv2e = myeri.reshape((mol.nao**2, -1))
e, v = scipy.linalg.eigh(Vij)
#print(e)
ltensor = cholesky(myeri, mol.nao)
print(ltensor.shape)
print(l_tensor.shape)
print(np.linalg.norm(ltensor - l_tensor[:172,:,:]))
#print(l_tensor - ltensor)
eri_reconstruct = np.einsum('apq, ars->pqrs', ltensor, ltensor)
print(eri_reconstruct- eri)
print(np.linalg.norm(eri_reconstruct - eri))

(19, 19, 19, 19)
(19, 19, 19, 19)
2.4121010810813428e-14
Vij =  [[ 1.05876541e+00  2.74324214e-02 -4.04077617e-17 ... -1.72963807e-18
  -6.39103296e-17  4.42200254e-01]
 [ 2.74324214e-02  6.18641785e-03  4.62804111e-18 ...  4.89227820e-18
  -2.67277392e-18  1.37378388e-02]
 [-4.04077617e-17  4.62804111e-18  1.31470352e-01 ... -1.28288925e-18
  -7.42604814e-03 -2.68944502e-17]
 ...
 [-8.79348322e-17  3.14838020e-17 -1.28288925e-18 ...  5.88863179e-02
   6.64802523e-18 -9.92214143e-17]
 [-6.39103296e-17 -2.67277392e-18 -7.42604814e-03 ...  6.64802523e-18
   5.09611613e-02 -1.19911065e-16]
 [ 4.42200254e-01  1.37378388e-02 -2.68944502e-17 ... -1.83419997e-17
  -1.19911065e-16  9.84857028e-01]]
e =  [-6.49800662e-16 -5.45171942e-16 -2.21958658e-16 -1.90786528e-16
 -1.70933867e-16 -1.07543870e-16 -5.33062919e-17 -4.55769836e-17
 -3.37752723e-17 -2.93647127e-17 -2.69127511e-17 -2.66046209e-17
 -2.65912344e-17 -2.46319435e-17 -2.42128037e-17 -2.05220050e-17
 -1.91696676e-17 -1.64036581e-17 -1

/Users/formic/pyscf/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [2]:
#Need to check the difference between SVD and the eigenvalue decomposition
# SVD
u1, s1, v1 = scipy.linalg.svd(Vij)
#eigendecomposition
e2, v2 = scipy.linalg.eigh(Vij)

#check the difference between the two decompositions
print(np.linalg.norm(np.diag(s1 - np.flip(e2))))
#print(np.linalg.norm(Vij - v2 @ np.diag(e2) @ np.linalg.inv(v2)))
#print(np.linalg.norm(Vij - u1 @ np.diag(s1) @ v1))
print(np.linalg.norm(u1 @ np.diag(s1) @ v1 - v2 @ np.diag(e2) @ np.linalg.inv(v2)))
print(np.linalg.norm(u1 - v1.T))
print(u1 - np.flip(v2, 1))
print(np.linalg.norm(u1 - v1.T))
print(np.trace(u1) - np.trace(np.flip(v2, 1)))
print(np.linalg.norm(v1.T - np.flip(v2, 1)))
#Conclusion: the order of eignevalues is different between the two decompositions, the values are the same

1.7660293973601188e-14
8.034508479872029e-14
19.32731313478013
[[-1.11022302e-16  9.99200722e-16  1.60982339e-15 ... -2.48782482e-18
   1.50991772e-19 -9.14098743e-08]
 [-2.22044605e-16  2.77555756e-16 -1.91513472e-15 ... -2.22441279e-01
  -2.10585025e-01  4.31925962e-01]
 [-1.00613962e-16  2.22044605e-16 -8.88178420e-16 ...  5.12187050e-02
   9.11140655e-02  3.41759039e-01]
 ...
 [ 1.32481644e-18  3.45994057e-19  1.13029293e-17 ... -2.75218262e-01
  -1.63606870e-02 -9.42130378e-03]
 [ 1.40771647e-20 -1.39428096e-18 -6.63563447e-19 ... -3.34799108e-04
  -7.08891347e-01  1.29065455e-03]
 [-1.11022302e-16  4.02455846e-16  2.49800181e-16 ...  1.00160513e-09
  -3.51707886e-09 -8.31723929e-09]]
19.32731313478013
0.13976972981337588
26.458056915934627


In [6]:
vbar = np.random.rand(ltensor.shape[0])
print(vbar)
print(np.array([vbar[i]* np.eye(ltensor.shape[1]) for i in range(ltensor.shape[0])]).shape)

[0.62851229 0.31993644 0.07151968 0.32080744 0.2490911  0.02693654
 0.47060253 0.26103419 0.29568246 0.3304064  0.33262833 0.34677683
 0.81354129 0.24422372 0.96412104 0.16024533 0.13066467 0.75375837
 0.86961922 0.25139311 0.41774781 0.65697889 0.56162414 0.55268638
 0.303971   0.14623251 0.42055597 0.34653442 0.07858803 0.76201811
 0.85404764 0.46992904 0.1171749  0.06851951 0.75443421 0.26898944
 0.24106089 0.71414801 0.07146952 0.26512224 0.9399964  0.52649572
 0.84039265 0.90935582 0.12869115 0.67305631 0.71464897 0.0292356
 0.13395809 0.55319536 0.79062233 0.63815166 0.19789944 0.38556169
 0.3550356  0.96773696 0.55517069 0.2765426  0.27734669 0.19473554
 0.0184072  0.89300683 0.46926053 0.26247743 0.03988516 0.12936127
 0.66176256 0.31776328 0.4082722  0.29374835 0.24619005 0.22133191
 0.52583893 0.91055096 0.46938465 0.93889492 0.61443872 0.07209679
 0.13928477 0.67773831 0.11208693 0.32784638 0.43514773 0.66694131
 0.99203658 0.62341126 0.17794008 0.70139774 0.08538469 0.50575

In [2]:
mol = gto.M(
    atom=[("H", 0, 0, 0), ("H", 1.6, 0, 0)],
    basis='sto-6g',
    unit='Bohr',
    verbose=5
)

ovlp_mat = mol.intor('int1e_ovlp')
X = lo.orth.lowdin(ovlp_mat)
myeri = mol.ao2mo(X)
myeri = ao2mo.restore(1, np.asarray(myeri), mol.nao)
ltensor = cholesky(myeri, mol.nao)
print(mol.nao)
print(ltensor)
eri_reconstruct = np.einsum('apq, ars->pqrs', ltensor, ltensor)
print(eri_reconstruct- myeri)
print(np.linalg.norm(eri_reconstruct - myeri))

System: uname_result(system='Darwin', node='Jinghongs-MacBook-Pro.local', release='22.6.0', version='Darwin Kernel Version 22.6.0: Wed Jul  5 22:21:53 PDT 2023; root:xnu-8796.141.3~6/RELEASE_ARM64_T6020', machine='arm64')  Threads 1
Python 3.11.4 (main, Jul  5 2023, 08:54:11) [Clang 14.0.6 ]
numpy 1.24.3  scipy 1.10.1
Date: Wed Sep 27 23:43:54 2023
PySCF version 2.3.0
PySCF path  /Users/formic/pyscf
GIT HEAD (branch master) 8d4d6926e8b0f24ddf33e3b02a3c9052238b1d17

[CONFIG] conf_file None
[INPUT] verbose = 5
[INPUT] max_memory = 4000 
[INPUT] num. atoms = 2
[INPUT] num. electrons = 2
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = Bohr
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 H     